In [14]:
import pandas as pd
import numpy as np
pd.options.display.max_columns = 99

### Reading in first 5 rows to inspect data:

In [5]:
df = pd.read_csv("loans_2007.csv", nrows=5)
df

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,last_credit_pull_d,collections_12_mths_ex_med,policy_code,application_type,acc_now_delinq,chargeoff_within_12_mths,delinq_amnt,pub_rec_bankruptcies,tax_liens
0,1077501,1296599.0,5000.0,5000.0,4975.0,36 months,10.65%,162.87,B,B2,NaN,10+ years,RENT,24000.0,Verified,Dec-2011,Fully Paid,n,credit_card,Computer,860xx,AZ,27.65,0.0,Jan-1985,1.0,3.0,0.0,13648.0,83.7%,9.0,f,0.00,0.00,5863.155187,5833.84,5000.00,863.16,0.00,0.00,0.00,Jan-2015,171.62,Jun-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
1,1077430,1314167.0,2500.0,2500.0,2500.0,60 months,15.27%,59.83,C,C4,Ryder,< 1 year,RENT,30000.0,Source Verified,Dec-2011,Charged Off,n,car,bike,309xx,GA,1.00,0.0,Apr-1999,5.0,3.0,0.0,1687.0,9.4%,4.0,f,0.00,0.00,1008.710000,1008.71,456.46,435.17,0.00,117.08,1.11,Apr-2013,119.66,Sep-2013,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
2,1077175,1313524.0,2400.0,2400.0,2400.0,36 months,15.96%,84.33,C,C5,NaN,10+ years,RENT,12252.0,Not Verified,Dec-2011,Fully Paid,n,small_business,real estate business,606xx,IL,8.72,0.0,Nov-2001,2.0,2.0,0.0,2956.0,98.5%,10.0,f,0.00,0.00,3005.666844,3005.67,2400.00,605.67,0.00,0.00,0.00,Jun-2014,649.91,Jun-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
3,1076863,1277178.0,10000.0,10000.0,10000.0,36 months,13.49%,339.31,C,C1,AIR RESOURCES BOARD,10+ years,RENT,49200.0,Source Verified,Dec-2011,Fully Paid,n,other,personel,917xx,CA,20.00,0.0,Feb-1996,1.0,10.0,0.0,5598.0,21%,37.0,f,0.00,0.00,12231.890000,12231.89,10000.00,2214.92,16.97,0.00,0.00,Jan-2015,357.48,Apr-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
4,1075358,1311748.0,3000.0,3000.0,3000.0,60 months,12.69%,67.79,B,B5,University Medical Group,1 year,RENT,80000.0,Source Verified,Dec-2011,Current,n,other,Personal,972xx,OR,17.94,0.0,Jan-1996,0.0,15.0,0.0,27783.0,53.9%,38.0,f,461.73,461.73,3581.120000,3581.12,2538.27,1042.85,0.00,0.00,0.00,Jun-2016,67.79,Jun-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0


### Calculating the memory usage of 1000 rows in MB:

In [7]:
df = pd.read_csv("loans_2007.csv", nrows=1000)
df.memory_usage(deep=True).sum() / 1024 ** 2

1.5502090454101562

### Determining how many rows we can pull to stay under 5 MB:

In [10]:
memory = 0 
rowcount = 1000
while memory < 5:
    df = pd.read_csv("loans_2007.csv", nrows = rowcount)
    oldmemory = memory
    memory = df.memory_usage(deep=True).sum() / 1024 ** 2
    rowcount += 100

print(oldmemory, rowcount)

4.959044456481934 3400


### Find total rows in data set and start data processing in chunks:

In [13]:
c_iter = pd.read_csv("loans_2007.csv", chunksize=3400)
numrows = 0
for c in c_iter:
    numrows += len(c)
print(numrows)

42538


### Find out how many columns have a numeric and string types:

In [17]:
c_iter = pd.read_csv("loans_2007.csv", chunksize=3400)
nums = []
strings = []
for c in c_iter:
    nums.append(c.select_dtypes(include=[np.number]).shape[1])
    strings.append(c.select_dtypes(include=["object"]).shape[1])
print(nums)
print(strings)

[31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 30, 30]
[21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 22, 22]


### How many unique values are in each string column?

In [44]:
c_iter = pd.read_csv("loans_2007.csv", chunksize=3400)
unique_counts = {}
for c in c_iter:
    str_objs = c.select_dtypes(include=["object"])
    columns = str_objs.columns
    for col in columns:
        if col in unique_counts:
            unique_counts[col] += c[col].value_counts()
        else:
            unique_counts[col] = c[col].value_counts()
unique_counts


{'addr_state': CA    7429.0
 NY    4065.0
 FL    3104.0
 TX    2915.0
 NJ    1988.0
 IL    1672.0
 GA    1503.0
 PA    1651.0
 VA    1487.0
 OH    1329.0
 WA     888.0
 MA    1438.0
 NC       NaN
 MD    1125.0
 AZ     933.0
 MO     765.0
 CT     816.0
 CO     857.0
 MN     652.0
 WI     516.0
 OR     468.0
 MI     796.0
 NV     527.0
 AL     484.0
 LA     461.0
 SC     489.0
 UT     278.0
 KY     359.0
 AR     261.0
 OK     317.0
 KS       NaN
 HI     181.0
 RI     208.0
 DC     224.0
 AK      86.0
 NH     188.0
 SD      67.0
 WV     187.0
 NM     205.0
 MT      96.0
 VT      57.0
 DE     136.0
 WY      87.0
 Name: addr_state, dtype: float64, 'application_type': INDIVIDUAL    42535
 Name: application_type, dtype: int64, 'earliest_cr_line': Oct-2000    370.0
 Apr-2000    266.0
 Nov-1998    390.0
 Nov-2000    340.0
 Jan-2000    300.0
 May-2000    265.0
 Nov-1999    337.0
 Dec-1998    366.0
 Oct-1999    393.0
 Feb-2000    239.0
 Oct-2003    264.0
 Sep-1998    267.0
 Aug-2000    283.0
 Dec

### Find string columns that contain less than 50% unique values to identify candidates for conversion to category data type:

In [65]:
category_columns = []

for col in unique_counts.keys():
    #print(col)
    #print(unique_counts[col].size)
    #print(unique_counts[col].sum())
    unique_count = unique_counts[col].size
    total_count = unique_counts[col].sum()
    
    if total_count != 0 and unique_count / total_count < .5:
        category_columns.append((col, unique_count))
        
for row in category_columns:
    print(row)

('verification_status', 3)
('emp_length', 11)
('revol_util', 919)
('pymnt_plan', 1)
('grade', 7)
('last_credit_pull_d', 55)
('sub_grade', 35)
('home_ownership', 3)
('earliest_cr_line', 378)
('purpose', 13)
('addr_state', 43)
('zip_code', 593)
('title', 1577)
('last_pymnt_d', 55)
('application_type', 1)
('initial_list_status', 1)
('term', 2)


### Find which floats have no missing values and could be candidates to convert to integers:

In [70]:
c_iter = pd.read_csv("loans_2007.csv", chunksize=3400)

null_counts_combined = []
for c in c_iter:
    null_counts_combined.append(c.select_dtypes(include=["float"]).isnull().sum())
null_counts_overall = pd.concat(null_counts_combined)
null_counts_overall.groupby(null_counts_overall.index).sum()
    

acc_now_delinq                  32
annual_inc                       7
chargeoff_within_12_mths       148
collection_recovery_fee          3
collections_12_mths_ex_med     148
delinq_2yrs                     32
delinq_amnt                     32
dti                              3
funded_amnt                      3
funded_amnt_inv                  3
inq_last_6mths                  32
installment                      3
last_pymnt_amnt                  3
loan_amnt                        3
member_id                        3
open_acc                        32
out_prncp                        3
out_prncp_inv                    3
policy_code                      3
pub_rec                         32
pub_rec_bankruptcies          1368
recoveries                       3
revol_bal                        3
tax_liens                      108
total_acc                       32
total_pymnt                      3
total_pymnt_inv                  3
total_rec_int                    3
total_rec_late_fee  

### Calculate the total memory usage across all chunks:

In [74]:
c_iter = pd.read_csv("loans_2007.csv", chunksize=3400)
mem_list = []
for c in c_iter:
    mem_list.append(c.memory_usage(deep=True).sum())
sum(mem_list) / 1024 ** 2

66.29912662506104